## Import des modules

In [126]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Selection
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    cross_validate,
)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.inspection import permutation_importance

#Preprocess
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

#Modèles
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

bc = pd.read_csv('seatle_after_analyze.csv')

# Traitement des Types de Batiments
- PrimaryPropertyType
- Méthode pour numériser des catégories non ordonnées >>> LabelEncoder
- Méthode pour numériser des catégories non ordonnées >>> one hot encoding

In [127]:
Column_To_Encode = 'PrimaryPropertyType'
Label_Encode = 'LabelEncoder' + Column_To_Encode

LabelEncoder = LabelEncoder()
bc[Label_Encode] = LabelEncoder.fit_transform(bc[Column_To_Encode])
#print(bc[[Column_To_Label_Encode, 'LabelEncoderColumn']].head())
unique_pairs = bc[[Column_To_Encode, Label_Encode]].drop_duplicates().sort_values(Label_Encode)
print(unique_pairs)

              PrimaryPropertyType  LabelEncoderPrimaryPropertyType
134           Distribution Center                                0
1022                     Hospital                                1
2                           Hotel                                2
1                     K-12 School                                3
356                    Laboratory                                4
104                  Large Office                                5
335          Low-Rise Multifamily                                6
207                Medical Office                                7
112            Mixed Use Property                                8
1248                       Office                                9
0                           Other                               10
219        Refrigerated Warehouse                               11
54                 Residence Hall                               12
355                    Restaurant                             

# Traitement de la Localisation des Batiments
- Neighborhood
- Méthode pour numériser des catégories non ordonnées >>> one hot encoding

In [128]:
Column_To_Encode = 'Neighborhood'
Label_Encode = 'LabelEncoder' + Column_To_Encode

bc[Label_Encode] = LabelEncoder.fit_transform(bc[Column_To_Encode])
#print(bc[[Column_To_Label_Encode, 'LabelEncoderColumn']].head())
unique_pairs = bc[[Column_To_Encode, Label_Encode]].drop_duplicates().sort_values(Label_Encode)
print(unique_pairs)

             Neighborhood  LabelEncoderNeighborhood
9                 BALLARD                         0
22                CENTRAL                         1
62               DELRIDGE                         2
0                DOWNTOWN                         3
20                   EAST                         4
10       GREATER DUWAMISH                         5
5              LAKE UNION                         6
8   MAGNOLIA / QUEEN ANNE                         7
55                  NORTH                         8
2               NORTHEAST                         9
37              NORTHWEST                        10
1               SOUTHEAST                        11
68              SOUTHWEST                        12


# Traitement des Usages des Batiments

In [129]:
#ohe = OneHotEncoder(sparse_output=False)  # Newer versions
#labels = ohe.fit_transform(bc[["ListOfAllPropertyUseTypes"]])
#bc_ohe = pd.DataFrame(labels, columns=ohe.get_feature_names_out(["ListOfAllPropertyUseTypes"]))
#print(bc_ohe)

# APPROCHE 1 : Transformer les données pour OneHotEncoder
print("="*50)
print("APPROCHE 1 : TRANSFORMATION DES DONNÉES")
print("="*50)

def prepare_data_for_onehot(df, column, delimiter=','):
    """
    Prépare les données pour utiliser OneHotEncoder avec des valeurs multiples
    """
    # Créer une liste de toutes les combinaisons
    expanded_data = []
    original_indices = []

    for idx, values_str in df[column].items():
        if pd.notna(values_str) and values_str.strip():
            # Séparer les valeurs
            values = [v.strip() for v in values_str.split(delimiter) if v.strip()]

            # Créer une ligne pour chaque valeur
            for value in values:
                expanded_data.append(value)
                original_indices.append(idx)

    # Créer un DataFrame expandé
    expanded_df = pd.DataFrame({
        'original_index': original_indices,
        'category': expanded_data
    })

    return expanded_df

# Préparer les données
expanded_df = prepare_data_for_onehot(bc, 'ListOfAllPropertyUseTypes')

# --- Étape 1 : compter les occurrences
counts = expanded_df.sum()

# --- Étape 2 : définir les catégories fréquentes
frequent_categories = counts[counts > 10].index

# --- Étape 3 : remplacer les catégories rares par "Autre"
expanded_df["ListOfAllPropertyUseTypes_clean"] = expanded_df["category"].apply(
    lambda x: x if x in frequent_categories else "Autre"
)

# Utiliser OneHotEncoder sur les données expandues
encoder = OneHotEncoder(sparse_output=False)
encoded_expanded = encoder.fit_transform(expanded_df[['category']])

# Obtenir les noms des features
feature_names = encoder.get_feature_names_out(['category'])
encoded_df_expanded = pd.DataFrame(encoded_expanded, columns=feature_names)
encoded_df_expanded['original_index'] = expanded_df['original_index']

# Regrouper par index original pour recombiner les valeurs multiples
final_encoded = encoded_df_expanded.groupby('original_index')[feature_names].sum()
# Convertir en binaire (0 ou 1) car sum peut donner des valeurs > 1
final_encoded = (final_encoded > 0).astype(int)

#print("Résultat final regroupé :")
print(final_encoded)
#print()
category_counts = final_encoded.sum(axis=0).sort_values(ascending=False)
print(category_counts)




APPROCHE 1 : TRANSFORMATION DES DONNÉES
      original_index                               category
0                  0  Other - Entertainment/Public Assembly
1                  1                            K-12 School
2                  2                                  Hotel
3                  2                                Parking
4                  3                  Automobile Dealership
...              ...                                    ...
2229            1256         Fitness Center/Health Club/Gym
2230            1256                           Food Service
2231            1256                                 Office
2232            1256                     Other - Recreation
2233            1256                     Pre-school/Daycare

[2234 rows x 2 columns]
original_index                                              1398094
category          Other - Entertainment/Public AssemblyK-12 Scho...
dtype: object


TypeError: '>' not supported between instances of 'str' and 'int'